# Assignment 1
#### Extracting linguistic features using ```spaCy```

In [28]:
# import modules 
import spacy
import pandas as pd 
import os
import re
import string


# loading the spacy model
# define pipeline
nlp = spacy.load("en_core_web_md")


/tmp/ipykernel_3695/1459457600.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


##### ```Preprocessing``` function

In [21]:
# looking at the dataframe it seems the text has some spaces enoced as "\n". We want to replace these with spaces. 
# we also want to get rid of the doc.id and title between the brackets 

# make function to clean the text
def clean_text(text):
    """
    This function removes extra white spaces "\n" and characters between < and >, and trailing whitespaces (.strip). 
    """
    cleaned_text = re.sub(r'<.*?>|\s+', ' ', text).strip()

    return cleaned_text


'jg hedder ida   ... --'

### Get relative frequency of ```Nouns, Verbs, Adjective, and Adverbs``` per 10,000 words
### And get total number of *unique* PER, LOC, ORGS

In [51]:

# one long function 

def processing(path):    
    # open it and read it 
    with open(path, encoding="latin-1") as f:
        text = f.read()
    
    doc = nlp(text)

    #doc = clean_text(doc)

    # see some attributes 
    counts_per_pos = {"NOUN": 0, "ADV": 0, "ADJ": 0, "VERB": 0} # make dictionary 

    for token in doc: 
        if token.pos_ in counts_per_pos: # only take NOUN, ADV, ADj and VERB
                # get text and label 
            counts_per_pos[token.pos_] += 1


    counts_per_label = {"PERSON": 0, "LOC": 0, "ORG": 0}

    # docs is a list of spaCy Doc objects
    for entity in doc.ents:
        if entity.label_ in counts_per_label:
            # Increment the count for each label
            counts_per_label[entity.label_] += 1

    
    data = counts_per_label | counts_per_pos # merge the two dictionaries 
        

    return data



#make relative...; 
#  counts = df["pos"].value_counts()
#                relative_freqs_per_10000 = (counts / len(df)) * 10000

### Loop over each text file in the folder called ```in```

In [4]:
# get current working directory 
os.getcwd()

'/work/CDS-language/CDS-language/Assignments/Assignment1_lang/src'

In [53]:
# define paths
file_path = os.path.join(
    "..", 
    "in",
    "USEcorpus")

output_path = os.path.join(
    "..", 
    "out")

dirs = sorted(os.listdir(file_path))


# loop through the paths 
for directory in dirs: 
    subfolder = os.path.join(file_path, directory) # path.join instead of "datapath + "/" + directory"
    filenames = sorted(os.listdir(subfolder))

    corpus_texts = [] # make empty list to append the texts 

    for text_file in filenames:
        path = os.path.join(subfolder,text_file)
        data = processing(path)
        
        corpus_texts.append({'File': text_file, 'Folder': directory, **data}) # use dictionary, so it's easier to convert to df with the folder and file name


        # Convert the list of dictionaries to a pandas DataFrame
    corpus_df = pd.DataFrame(corpus_texts)
    df.to_csv(os.path.join(output_path, f"{directory}_spacy.csv"), index = False)





In [54]:
corpus_df

,File,Folder,PERSON,LOC,ORG,NOUN,ADV,ADJ,VERB
0,0140.c1.txt,c1,78,0,8,386,99,116,229
1,0165.c1.txt,c1,52,0,6,429,70,143,201
2,0200.c1.txt,c1,79,0,11,234,101,129,203
3,0219.c1.txt,c1,72,0,6,208,73,85,147
4,0238.c1.txt,c1,62,0,4,140,37,51,149
5,0501.c1.txt,c1,54,0,6,179,62,67,149
6,0502.c1.txt,c1,40,0,26,207,64,68,191


##### ```Preprocessing``` functions 


### For each sub-folder (a1, a2, a3, ...) save a table which shows the following information:
| Filename  | RelFreq NOUN | RelFreq VERB | RelFreq ADJ | RelFreq ADV | Unique PER | Unique LOC | Unique ORG |
|-----------|--------------|--------------|-------------|-------------|------------|------------|------------|
| file1.txt | ---          | ---          | ---         | ---         | ---        | ---        | ---        |
| file2.txt | ---          | ---          | ---         | ---         | ---        | ---        | ---        |
| etc       | ---          | ---          | ---         | ---         | ---        | ---        | ---        |
